# News Recommender: Natural Language Processing

*insert title page* with links

## CLEANING AND VECTORIZATION
In the first section of this notebook, we will be pre-processing, cleaning, lemmatizing, tagging, and vectorizing our webscraped tweets. The goal is to create an optimized document-term matrix for topic modeling.

In [19]:
import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from langdetect import detect
import unicodedata

### Standardize Encoding

Read in CSV as dataframe, and standardize encoding.

In [12]:
df_raw = pd.read_csv('df_raw.csv', encoding='utf-8', index_col=0)

In [5]:
df_raw.columns

Index(['user', 'date', 'url', 'outlinks', 'content'], dtype='object')

In [13]:
#reindex the columns for easier viewing
cols = df_raw.columns.tolist()

cols.insert(2, cols.pop(cols.index('url')))

df_raw= df_raw.reindex(columns= cols)

Decode our webscrapped tweets into ascii so we can remove emojis and foreign characters easily during our pre-processing steps.

In [29]:
#decoded unicode into ascii
df_raw['clean'] = df_raw['content'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))

In [30]:
df_raw.head()

,user,date,url,outlinks,content,clean
0,"{'username': 'TheAtlantic', 'displayname': 'Th...",2020-11-01T01:08:48+00:00,https://twitter.com/TheAtlantic/status/1322707...,['http://on.theatln.tc/YXH6gyR'],The Atlantic Daily: Will this decade be the ne...,The Atlantic Daily: Will this decade be the ne...
1,"{'username': 'TheAtlantic', 'displayname': 'Th...",2020-11-01T00:38:45+00:00,https://twitter.com/TheAtlantic/status/1322699...,['http://on.theatln.tc/l89Uzv7'],There's plenty that's going wrong for Trump. H...,There's plenty that's going wrong for Trump. H...
2,"{'username': 'TheAtlantic', 'displayname': 'Th...",2020-11-01T00:06:48+00:00,https://twitter.com/TheAtlantic/status/1322691...,['http://on.theatln.tc/ZGvkM7u'],"If Trump tries to steal the election, people w...","If Trump tries to steal the election, people w..."
3,"{'username': 'TheAtlantic', 'displayname': 'Th...",2020-10-31T23:34:45+00:00,https://twitter.com/TheAtlantic/status/1322683...,['http://on.theatln.tc/kypt5Zc'],The Trump campaign's “election-security operat...,The Trump campaign's election-security operati...
4,"{'username': 'TheAtlantic', 'displayname': 'Th...",2020-10-31T23:04:31+00:00,https://twitter.com/TheAtlantic/status/1322675...,['http://on.theatln.tc/rNbarVc'],"Even if Joe Biden wins decisively next week, t...","Even if Joe Biden wins decisively next week, t..."


### Preprocessing

In [32]:
def preprocess(tweet):
    """
    Takes in tweet and performs initial text cleaning/preprocessing.
    """
    #make sure doc is string
    tweet=str(tweet)
    #get rid of urls
    rem_url=re.sub(r'http\S+', '', tweet)
    #gets rid of @ tags
    rem_tag = re.sub('@\S+', '', rem_url)
    #gets rid of # in hashtag but keeps content of hashtag
    rem_hashtag = re.sub('#', '', rem_tag)
    #gets rid of special characters, numbers, etc.
    clean_text = re.sub(r'[^A-Za-z\s]','', rem_hashtag)

    return clean_text

In [33]:
df_raw['clean']=df_raw['clean'].map(preprocess)

In [80]:
df_raw[['content', 'clean']].head(3)

,content,clean
0,The Atlantic Daily: Will this decade be the ne...,The Atlantic Daily Will this decade be the new s
1,There's plenty that's going wrong for Trump. H...,Theres plenty thats going wrong for Trump Here...
2,"If Trump tries to steal the election, people w...",If Trump tries to steal the election people wi...


### Standardize Language

Let's remove any foreign language tweets to make sure we're only focusing on English. 

In [37]:
def english_only(x):
    """
    Take tweet, detect language, and only return English tweets, coding foreign language tweets as NaNs.
    """
    try:
        if detect(x) == 'en':
            return x
        else:
            return np.nan
    except:
        pass


In [38]:
%%time
#  Remove any non english tweets
df_raw['clean'] = df_raw['clean'].apply(lambda x: english_only(x))


CPU times: user 12min 4s, sys: 6.02 s, total: 12min 10s
Wall time: 12min 14s


In [39]:
df_raw.clean.isnull().sum()


1251

In [40]:
#drop non-English tweets
df_raw = df_raw[df_raw.clean.notnull()]

### Tagging and Lemmatizing

We only want the nouns (and proper nouns) in each tweet for topic modeling, as they are the essence of article subjects. Let's tag the nouns and return the lemmatized versions of tehm in a single step.

Because we also have so much data, we will incoporate the NLP pipeline in order to shorten the processing time. Tips on how to do this were found here: https://towardsdatascience.com/turbo-charge-your-spacy-nlp-pipeline-551435b664ad

In [41]:
nlp = spacy.load('en_core_web_sm', disable=[ 'parser', 'ner'])

In [42]:
def noun_lemmatize_pipe(doc):
    """
    Takes in tweet and returns only the lemmatized version of nouns (including proper nouns).
    """
    lemma_list = [token.lemma_ for token in doc
                  if token.pos_ == "NOUN" or token.pos_ =="PROPN"] 
    return lemma_list

#create a pipeline in order to shorten processing time
def preprocess_pipe(texts):
    """
    Inputs noun_lemmative_pipe function into NLP pipeline for faster processing.
    """
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=50):
        preproc_pipe.append(noun_lemmatize_pipe(doc))
    return preproc_pipe

In [43]:
%%time
#apply function and create a new column to house the outputs
df_raw['clean_lemmatized'] = preprocess_pipe(df_raw['clean'])


CPU times: user 2min 22s, sys: 670 ms, total: 2min 23s
Wall time: 2min 24s


In [79]:
df_raw[['content', 'clean', 'clean_lemmatized']].head(3)

,content,clean,clean_lemmatized
0,The Atlantic Daily: Will this decade be the ne...,The Atlantic Daily Will this decade be the new s,"[decade, s]"
1,There's plenty that's going wrong for Trump. H...,Theres plenty thats going wrong for Trump Here...,"[plenty, Trump, thing, campaign, gap, Joe, Bid..."
2,"If Trump tries to steal the election, people w...",If Trump tries to steal the election people wi...,"[Trump, election, people, coup, strategy, write]"


We've successfully filtered out the nouns and proper nouns, lemmitized them, while making sure our function runs on optimized time!

### Remove Additional Words

Let's filter out any additional words that may appear in the tweets but aren't related to article subjects, like the names of the publications and common headline section titles.

In [54]:
removal_words= ["Times", "Wall", "Street", "Journal", "New", "Yorker", "York", "Medium", "Wired", "Financial", "Washington", "Post", "Business", "Insider", "Economist", "The", "Atlantic", "Daily", "Weekly", "SPONSORED", "Sponsored", "BREAKING", "Breaking", "NEWS", "News" ]

In [73]:
df_raw['clean_lemmatized'] = df_raw['clean_lemmatized'].apply(lambda x: [word for word in x if word not in removal_words])

### Vectorize
Now let's rejoin our twice cleaned, lemmatized list of nouns and pronouns!

In [74]:
df_raw['clean_final'] = df_raw['clean_lemmatized'].apply(lambda x: ' '.join(x))

In [59]:
df_raw[['content', 'clean', 'clean_lemmatized', 'clean_final']].head(3)

,content,clean,clean_lemmatized,clean_final
0,The Atlantic Daily: Will this decade be the ne...,The Atlantic Daily Will this decade be the new s,"[decade, s]",decade s
1,There's plenty that's going wrong for Trump. H...,Theres plenty thats going wrong for Trump Here...,"[plenty, Trump, thing, campaign, gap, Joe, Bid...",plenty Trump thing campaign gap Joe Biden report
2,"If Trump tries to steal the election, people w...",If Trump tries to steal the election people wi...,"[Trump, election, people, coup, strategy, write]",Trump election people coup strategy write


We will be using TFID Vectorizer as opposed to Count Vectorizer, so we can give equal weight to rare words. Because our focus is on nouns and pronouns, rare words are likely to be just as, if not more impactful, as words that are frequently used. 

We'll also be using some of the built in parameters in the TFID Vectorizer as last checks before we output our doc-term matrix.

In [91]:
#define vectorizer and set parameters in order to standardize everything to lowercase, remove any stop words, and remove any word that appears below 0.005%, about 10 times.
tfidf = TfidfVectorizer(lowercase = True, stop_words= 'english', min_df = 0.00005)
#fit on fully cleaned dataframe column
doc_term_matrix = tfidf.fit_transform(df_raw['clean_final'])
#turn matrix into a dataframe with words as columns
matrix_df = pd.DataFrame(doc_term_matrix.toarray(), columns=tfidf.get_feature_names())

In [92]:
matrix_df

,aaron,ab,abandonment,abbey,abbott,abby,abc,abe,abenomics,aberration,...,zoo,zoological,zoom,zooms,zoos,zoox,zora,zuckerberg,zuckerbergs,zuzana
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Our doc-term matrix looks great! We have fully cleaned out all special characters and foreign language. We have extracted the nouns and pronouns in their lemmatized forms. Now we can topic model!


## TOPIC MODELING
In the second section of this notebook, we will be topic modeling.

In [96]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

### LSA

In [105]:
lsa = TruncatedSVD(10)
doc_topic = lsa.fit_transform(doc_term_matrix)
lsa.explained_variance_ratio_

array([0.00313351, 0.00424593, 0.00357008, 0.00321061, 0.00294962,
       0.00292731, 0.00283308, 0.00280643, 0.00272183, 0.00256971])

In [106]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2", "component_3", "component_4", "component_5", 
             "component_6", "component_7", "component_8", "component_9", "component_10"],
             columns = tfidf.get_feature_names())
topic_word

,aaron,ab,abandonment,abbey,abbott,abby,abc,abe,abenomics,aberration,...,zoo,zoological,zoom,zooms,zoos,zoox,zora,zuckerberg,zuckerbergs,zuzana
component_1,0.000,0.0,0.0,0.0,0.001,0.0,0.001,0.002,0.0,0.0,...,0.001,0.0,0.006,0.0,0.000,0.0,0.000,0.005,0.001,0.0
component_2,0.000,-0.0,0.0,0.0,0.000,-0.0,0.001,0.001,0.0,0.0,...,-0.001,-0.0,-0.002,-0.0,-0.000,-0.0,-0.000,0.001,0.000,0.0
component_3,0.000,0.0,0.0,0.0,-0.001,0.0,0.000,0.000,0.0,0.0,...,0.000,0.0,0.006,0.0,-0.000,0.0,0.000,0.007,0.000,0.0
component_4,-0.000,-0.0,-0.0,-0.0,0.000,0.0,-0.000,-0.001,-0.0,0.0,...,-0.000,-0.0,-0.001,-0.0,-0.000,-0.0,-0.000,-0.004,-0.001,0.0
component_5,0.000,0.0,0.0,-0.0,-0.001,0.0,-0.000,0.000,0.0,0.0,...,-0.000,-0.0,0.002,0.0,0.000,-0.0,-0.000,-0.001,0.000,0.0
component_6,0.000,-0.0,0.0,-0.0,0.001,0.0,-0.000,-0.000,0.0,0.0,...,-0.000,-0.0,0.000,0.0,0.001,-0.0,0.000,-0.001,-0.000,0.0
component_7,0.000,0.0,0.0,-0.0,0.000,0.0,-0.000,0.001,-0.0,-0.0,...,-0.000,-0.0,-0.001,0.0,-0.000,0.0,0.000,0.002,0.000,-0.0
component_8,0.000,-0.0,0.0,-0.0,0.000,0.0,-0.000,0.001,-0.0,0.0,...,-0.000,-0.0,-0.003,-0.0,-0.000,-0.0,0.000,0.001,-0.000,-0.0
component_9,0.001,-0.0,-0.0,0.0,0.000,-0.0,-0.000,0.001,0.0,-0.0,...,0.000,0.0,-0.001,0.0,-0.000,-0.0,-0.000,0.001,-0.000,-0.0
component_10,0.000,0.0,0.0,0.0,0.000,0.0,-0.001,0.001,-0.0,0.0,...,0.001,0.0,0.004,0.0,-0.000,0.0,0.001,0.005,0.000,0.0


In [107]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [109]:
display_topics(lsa, tfidf.get_feature_names(), 10)


Topic  0
coronavirus, people, trump, pandemic, president, year, world, time, case, opinion

Topic  1
trump, president, opinion, donald, biden, joe, trumps, election, campaign, house

Topic  2
people, year, time, life, world, way, thing, company, story, home

Topic  3
people, trump, coronavirus, president, opinion, case, donald, white, house, virus

Topic  4
pandemic, opinion, time, way, world, life, thing, trumps, woman, write

Topic  5
time, day, case, coronavirus, death, state, briefing, page, opinion, morning

Topic  6
page, look, world, edition, uk, thing, today, friday, monday, thursday

Topic  7
world, case, week, coronavirus, story, life, country, way, woman, outbreak

Topic  8
opinion, year, day, coronavirus, china, case, america, state, hong, page

Topic  9
way, day, story, life, thing, week, election, morning, hong, briefing


### NMF

In [112]:
nmf = NMF(10)
doc_topic = nmf.fit_transform(doc_term_matrix)

In [114]:
topic_word = pd.DataFrame(nmf.components_.round(3),
             index = ["component_1","component_2", "component_3", "component_4", "component_5", 
             "component_6", "component_7", "component_8", "component_9", "component_10"],
             columns = tfidf.get_feature_names())
topic_word

,aaron,ab,abandonment,abbey,abbott,abby,abc,abe,abenomics,aberration,...,zoo,zoological,zoom,zooms,zoos,zoox,zora,zuckerberg,zuckerbergs,zuzana
component_1,0.001,0.000,0.001,0.000,0.012,0.000,0.000,0.004,0.000,0.000,...,0.006,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000
component_2,0.001,0.000,0.000,0.000,0.003,0.000,0.006,0.004,0.000,0.001,...,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.008,0.002,0.000
component_3,0.002,0.000,0.000,0.003,0.000,0.000,0.003,0.004,0.001,0.000,...,0.003,0.001,0.014,0.000,0.000,0.001,0.001,0.019,0.002,0.001
component_4,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.001,...,0.003,0.000,0.026,0.000,0.000,0.001,0.001,0.016,0.000,0.001
component_5,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.001,0.000,0.028,0.001,0.001,0.000,0.000,0.002,0.001,0.000
component_6,0.001,0.000,0.001,0.000,0.000,0.000,0.001,0.003,0.001,0.000,...,0.000,0.000,0.014,0.000,0.003,0.000,0.000,0.007,0.002,0.001
component_7,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,...,0.000,0.000,0.001,0.000,0.000,0.000,0.001,0.008,0.001,0.000
component_8,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.007,0.000,0.000,...,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.018,0.001,0.000
component_9,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.003,0.000,...,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.005,0.000,0.000
component_10,0.001,0.000,0.000,0.001,0.001,0.000,0.000,0.002,0.000,0.000,...,0.002,0.000,0.018,0.000,0.000,0.000,0.003,0.031,0.001,0.001


In [115]:
display_topics(nmf, tfidf.get_feature_names(), 10)


Topic  0
coronavirus, case, death, state, outbreak, country, vaccine, crisis, infection, number

Topic  1
trump, president, donald, biden, joe, trumps, election, analysis, house, campaign

Topic  2
year, company, market, stock, month, business, report, school, government, tech

Topic  3
people, life, home, covid, way, thing, city, police, virus, work

Topic  4
pandemic, way, company, business, life, economy, job, worker, money, covid

Topic  5
time, thing, life, way, write, woman, money, home, child, crisis

Topic  6
page, look, edition, uk, friday, thursday, monday, today, tuesday, wednesday

Topic  7
world, country, way, china, war, today, week, company, climate, change

Topic  8
opinion, trumps, america, republicans, trump, way, court, crisis, republican, election

Topic  9
day, story, briefing, morning, election, week, hong, kong, end, europe


#### Next steps:
NMF works better than LDA

add words below to removal_list before vectorizing: way, page, edition, morning, monday, tuesday, weds, etc. , thing, briefing, day, story, year, life, write, wrote